In [76]:
import warnings
warnings.filterwarnings('ignore')

In [77]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [78]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [79]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [80]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [81]:
# Create our features
X = df.drop(columns=['loan_status'])
X = pd.get_dummies(X)

# Create our target
y = df[target]

In [82]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [83]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [84]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [85]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [86]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()

X_train_resampled, y_train_resampled = ros.fit_resample(X_train_scaled, y_train)

print(Counter(y_train_resampled.loan_status).items())

dict_items([('low_risk', 51345), ('high_risk', 51345)])


In [87]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=1)

rfc.fit(X_train_resampled, y_train_resampled)

y_preds_rf = rfc.predict(X_test_scaled)

In [88]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score as bas

bas(y_test, y_preds_rf)

0.6747080291970803

In [89]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix as cm

cm(y_test, y_preds_rf)

array([[   28,    52],
       [   10, 17115]])

In [90]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced as cri

print(cri(y_test, y_preds_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.74      0.35      1.00      0.47      0.59      0.33        80
   low_risk       1.00      1.00      0.35      1.00      0.59      0.37     17125

avg / total       1.00      1.00      0.35      1.00      0.59      0.37     17205



In [91]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
sorted(zip(rfc.feature_importances_, X.columns), reverse=True)

[(0.09041655977968574, 'total_rec_prncp'),
 (0.0731534987795952, 'total_pymnt'),
 (0.07145537636940631, 'total_pymnt_inv'),
 (0.06276340464611246, 'last_pymnt_amnt'),
 (0.059089730770691164, 'total_rec_int'),
 (0.03136567728041018, 'issue_d_Jan-2019'),
 (0.028565138796792653, 'issue_d_Mar-2019'),
 (0.027676212906845205, 'int_rate'),
 (0.02207697490755383, 'installment'),
 (0.019254221607457798, 'out_prncp_inv'),
 (0.019067228796115887, 'out_prncp'),
 (0.018240486732008763, 'total_rec_late_fee'),
 (0.01646224243212217, 'mo_sin_old_rev_tl_op'),
 (0.01589937326981533, 'annual_inc'),
 (0.015707413376210892, 'dti'),
 (0.01570478747026036, 'max_bal_bc'),
 (0.014020500533340278, 'total_bc_limit'),
 (0.01329412635899517, 'next_pymnt_d_May-2019'),
 (0.012870961130065727, 'total_bal_ex_mort'),
 (0.012828886022573942, 'tot_hi_cred_lim'),
 (0.012728686949191095, 'avg_cur_bal'),
 (0.012592147934836798, 'mo_sin_old_il_acct'),
 (0.012518059132798348, 'bc_util'),
 (0.01241863464813719, 'loan_amnt'),
 

### Easy Ensemble AdaBoost Classifier

In [92]:
# Train the Classifier
# YOUR CODE HERE
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(random_state=1)

abc.fit(X_train_resampled, y_train_resampled)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=1)

In [93]:
y_preds_abc = abc.predict(X_test_scaled)

In [94]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_preds_abc)

0.8722591240875912

In [95]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_preds_abc)

array([[   62,    18],
       [  522, 16603]])

In [96]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_preds_abc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.11      0.78      0.97      0.19      0.87      0.74        80
   low_risk       1.00      0.97      0.78      0.98      0.87      0.77     17125

avg / total       0.99      0.97      0.78      0.98      0.87      0.77     17205



In [97]:
sorted(zip(abc.feature_importances_, X.columns), reverse=True)

[(0.2, 'total_rec_int'),
 (0.14, 'last_pymnt_amnt'),
 (0.12, 'total_rec_prncp'),
 (0.08, 'issue_d_Mar-2019'),
 (0.08, 'issue_d_Jan-2019'),
 (0.08, 'installment'),
 (0.06, 'total_pymnt'),
 (0.06, 'next_pymnt_d_Apr-2019'),
 (0.04, 'int_rate'),
 (0.02, 'total_rec_late_fee'),
 (0.02, 'revol_bal'),
 (0.02, 'next_pymnt_d_May-2019'),
 (0.02, 'max_bal_bc'),
 (0.02, 'loan_amnt'),
 (0.02, 'il_util'),
 (0.02, 'annual_inc'),
 (0.0, 'verification_status_Verified'),
 (0.0, 'verification_status_Source Verified'),
 (0.0, 'verification_status_Not Verified'),
 (0.0, 'total_rev_hi_lim'),
 (0.0, 'total_pymnt_inv'),
 (0.0, 'total_il_high_credit_limit'),
 (0.0, 'total_cu_tl'),
 (0.0, 'total_bc_limit'),
 (0.0, 'total_bal_il'),
 (0.0, 'total_bal_ex_mort'),
 (0.0, 'total_acc'),
 (0.0, 'tot_hi_cred_lim'),
 (0.0, 'tot_cur_bal'),
 (0.0, 'tot_coll_amt'),
 (0.0, 'tax_liens'),
 (0.0, 'recoveries'),
 (0.0, 'pymnt_plan_n'),
 (0.0, 'pub_rec_bankruptcies'),
 (0.0, 'pub_rec'),
 (0.0, 'policy_code'),
 (0.0, 'percent_bc_gt

## Use the above to answer the following:

- Which model had the best balanced accuracy score?

#### AdaBoost model had much higher BAS than random forest.

- Which model had the best recall score?

#### Both score high on recall, but the RF model scored slightly higher.

- Which model had the best geometric mean score?

#### The model with the highest geometric mean was AdaBoost.

- What are the top three features?

#### Top three features for both models are given below.

In [105]:
print(f'''BALANCED ACCURACY SCORES:

Random forest classifier: \
{balanced_accuracy_score(y_test, y_preds_rf)}
AdaBoost classifier: \
{balanced_accuracy_score(y_test, y_preds_abc)}



''')

print(f'''CLASSIFICATION REPORTS:

Random forest classifier:
{classification_report_imbalanced(y_test, y_preds_rf)}

AdaBoost classifier:
{classification_report_imbalanced(y_test, y_preds_abc)}



''')

print(f'''TOP FEATURES:

RF:
{sorted(zip(rfc.feature_importances_, X.columns), reverse=True)[:3]}

ABC:
{sorted(zip(abc.feature_importances_, X.columns), reverse=True)[:3]}
''')

BALANCED ACCURACY SCORES:

Random forest classifier: 0.6747080291970803
AdaBoost classifier: 0.8722591240875912




CLASSIFICATION REPORTS:

Random forest classifier:
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.74      0.35      1.00      0.47      0.59      0.33        80
   low_risk       1.00      1.00      0.35      1.00      0.59      0.37     17125

avg / total       1.00      1.00      0.35      1.00      0.59      0.37     17205


AdaBoost classifier:
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.11      0.78      0.97      0.19      0.87      0.74        80
   low_risk       1.00      0.97      0.78      0.98      0.87      0.77     17125

avg / total       0.99      0.97      0.78      0.98      0.87      0.77     17205





TOP FEATURES:

RF:
[(0.09041655977968574, 'total_rec_prncp'), (0.0731534987795952, 'total_pymnt'), (0.07145537636940631, 'total_pymnt_inv